In [ ]:
# with thanks to Richard Chadwick, https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25
# much of the code for this scraper is either taken or adapted from his

In [1]:
# !pip install tweepy

In [7]:
import tweepy as tw
import datetime
import pandas as pd

In [8]:
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        'xOM5aibCz665Rz3Bq6hiOlXD0',
        'consumer_secret':     '9F5KTULDv9YkJY8jJXW4g735toRziLW13ZTWV7l33h3LAPDF4F',
        'access_token_key':    '20414579-PwO6nnfmFWfIvC4QNWvQMK25H63P8b61TINjby3Hj',
        'access_token_secret': 'pOJpLVA7yzSdSY4JBqytwwU4Pi28m0IAmw38KnCtiP8m7'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tw.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tw.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dril", #BECAUSE WHO ELSE!
                         mine_retweets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [9]:
miner=TweetMiner(result_limit = 200 )

In [10]:
mined_tweets = miner.mine_user_tweets(user='CPAC', max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [11]:
import time

handle_list = ['CPAC', 'RepMTG', 'laurenboebert', 'HawleyMO', 'RepAndyBiggsAZ', 
               'MattForMontana', 'SteveDaines', 'RepGosar', 'RepMoBrooks']

twitter_dict = {}
counter = 0

for name in handle_list:
    try:
        twitter_dict[name] = []
        twitter_dict[name].append(
            miner.mine_user_tweets(user=name, max_pages=17))
        counter = counter + 1
        if counter % 40 == 0:
            time.sleep(900)  # 15 minute sleep time
    # if name invalid print name and remove key
    except:
        print(name, 'is invalid or locked')
        twitter_dict.pop(name)

all_tweets = pd.concat([pd.DataFrame(twitter_dict[i][0])
                        for i in twitter_dict])

In [13]:
all_tweets.shape

(24255, 15)

In [14]:
all_tweets.set_index('created_at', inplace=True)

In [15]:
all_tweets.sort_index(ascending=True, inplace=True)

In [16]:
all_tweets.to_csv('./politicians1.csv')